<a href="https://colab.research.google.com/github/arad4228/BigData_and_A.I/blob/main/%EB%B9%85%EB%8D%B0%EC%9D%B4%ED%84%B0%20%EC%9D%91%EC%9A%A9(5%EC%9B%94%2019%EC%9D%BC%20%EC%8B%A4%EC%8A%B5).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdaBoost

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import matplotlib as plt

cdata = load_breast_cancer()

print(dir(cdata))
print(cdata.data.shape)
print(cdata.feature_names)
print(cdata.filename)
print(cdata.target)
print(cdata.target_names)

['DESCR', 'data', 'data_module', 'feature_names', 'filename', 'frame', 'target', 'target_names']
(569, 30)
['mean radius' 'mean texture' 'mean perimeter' 'mean area'
 'mean smoothness' 'mean compactness' 'mean concavity'
 'mean concave points' 'mean symmetry' 'mean fractal dimension'
 'radius error' 'texture error' 'perimeter error' 'area error'
 'smoothness error' 'compactness error' 'concavity error'
 'concave points error' 'symmetry error' 'fractal dimension error'
 'worst radius' 'worst texture' 'worst perimeter' 'worst area'
 'worst smoothness' 'worst compactness' 'worst concavity'
 'worst concave points' 'worst symmetry' 'worst fractal dimension']
breast_cancer.csv
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 1 1 1 0 1 0 0 1 1 1 1 0 1 0 0
 1 0 1 0 0 1 1 1 0 0 1 0 0 0 1 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 0 0 0 1 0 0 1 1 1 0 0 1 0 1 0 0 1 0 0 1 1 0 1 1 0 1 1 1 1 0 1
 1 1 1 1 1 1 1 1 0 1

In [ ]:
import numpy
X = cdata.data
X = X.astype(float)
y = numpy.where(cdata.target == 0, -1, 1)
print(X.shape)
print(y.shape)

trainX, testX, trainy, testy = train_test_split(X,y,test_size = 0.3, random_state =1)
print(trainX.shape)
print(testX.shape)

(569, 30)
(569,)
(398, 30)
(171, 30)


# AdaBoost -> 필요함수(1) Compute error

In [ ]:
def compute_error(y, y_pred, w_j):
  return (numpy.sum((w_j * numpy.not_equal(y, y_pred)), axis = -1).astype(int)) / numpy.sum(w_j)

# AdaBoost -> 필요함수(2) Compute alpha

In [ ]:
def compute_alpha(error):
  return 0.5*numpy.log((1-error+1e-6)/(error+1e-6))

# AdaBoost -> 필요함수(3) update weight

In [ ]:
def update_weights(w_j, alpha, y, y_pred):
  w_j_updated = numpy.where(numpy.not_equal(y,y_pred),
                            w_j * numpy.exp(alpha * (numpy.not_equal(y, y_pred))),
                            w_j * numpy.exp(-alpha * (numpy.not_equal(y, y_pred))))
            
  w_j_updated = w_j_updated / numpy.sum(w_j_updated)
  return w_j_updated

In [ ]:
class AdaBoost:

  def __init__(self):
    self.alphas = []
    self.slearner = []
    self.M = None
    self.training_errors = []
    self.prediction_errors = []

  def fit(self, X, y, M = 100):
    self.M = M
    # W_j 초기화 --> W_j: tensor --> (1,1,1,...1)
    w_j = numpy.array(numpy.ones(len(y)))/ len(y)
    for m in range(0,M):
      # 1. weak learner을 훈련
      wlearner = DecisionTreeClassifier(max_depth = 1)
      wlearner.fit(X, y, sample_weight = w_j)

      y_pred = wlearner.predict(X)

      self.slearner.append(wlearner)

      # 2. error계산
      error_m = compute_error(y, y_pred, w_j)
      self.training_errors.append(error_m)

      # 3. alpha계산
      alpha_m = compute_alpha(error_m)
      self.alphas.append(alpha_m)

      # 4. w_j를 update
      w_j = update_weights(w_j, alpha_m , y, y_pred)


  def predict (self, X):
    weak_preds = numpy.zeros(shape=(X.shape[0], len(self.slearner)))

    for i, m in enumerate(range(self.M)):
      y_pred_m = self.slearner[m].predict(X).astype(float) * self.alphas[m]
      weak_preds[:, m] = y_pred_m
    y_pred = numpy.sign(numpy.sum(weak_preds, axis =-1))
    return y_pred


In [ ]:
len_wlearner = 100
ada = AdaBoost()
ada.fit(trainX, trainy, M= len_wlearner)
y_pred = ada.predict(testX)

accuracy = (y_pred == testy).sum()/ len(testy)
print(accuracy)

0.9298245614035088


In [ ]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

tree_model = DecisionTreeClassifier(max_depth = 1)
ada = AdaBoostClassifier(base_estimator=tree_model, n_estimators=1000, random_state = 0)
ada.fit(trainX,trainy)
pred = ada.predict(testX)
print(accuracy_score(testy,pred))


0.9766081871345029


# GradientBoost


In [ ]:
from sklearn.datasets import load_diabetes
ddata = load_diabetes()

print(dir(ddata))
print(ddata.data.shape)
print(ddata.feature_names)
print(ddata.target)

['DESCR', 'data', 'data_filename', 'data_module', 'feature_names', 'frame', 'target', 'target_filename']
(442, 10)
['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']
[151.  75. 141. 206. 135.  97. 138.  63. 110. 310. 101.  69. 179. 185.
 118. 171. 166. 144.  97. 168.  68.  49.  68. 245. 184. 202. 137.  85.
 131. 283. 129.  59. 341.  87.  65. 102. 265. 276. 252.  90. 100.  55.
  61.  92. 259.  53. 190. 142.  75. 142. 155. 225.  59. 104. 182. 128.
  52.  37. 170. 170.  61. 144.  52. 128.  71. 163. 150.  97. 160. 178.
  48. 270. 202. 111.  85.  42. 170. 200. 252. 113. 143.  51.  52. 210.
  65. 141.  55. 134.  42. 111.  98. 164.  48.  96.  90. 162. 150. 279.
  92.  83. 128. 102. 302. 198.  95.  53. 134. 144. 232.  81. 104.  59.
 246. 297. 258. 229. 275. 281. 179. 200. 200. 173. 180.  84. 121. 161.
  99. 109. 115. 268. 274. 158. 107.  83. 103. 272.  85. 280. 336. 281.
 118. 317. 235.  60. 174. 259. 178. 128.  96. 126. 288.  88. 292.  71.
 197. 186.  25.  84.  96. 195.  53. 217.

GradientBoost -> Data 구축

In [ ]:
X = ddata.data
y = ddata.target
print(X.shape)
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state=1)

(442, 10)


GradientBoost -> GradientBoost class 정의

In [ ]:
from sklearn.tree import DecisionTreeRegressor

class GradientBoost:

  def __init__(self, learning_rate, n_estimators, max_depth = 1):
    self.learning_rate = learning_rate
    self.n_estimators = n_estimators
    self.max_depth = max_depth
    self.trees = []

  def fit(self, X, y):
    # 1. base classifer --> 평균
    self.f0 = y.mean()
    f_hat = self.f0

    for _ in range(self.n_estimators):
      # 1. residual 계산
      r = y - f_hat

      # 2. modify classifier 계산
      tree = DecisionTreeRegressor(max_depth =self.max_depth, random_state = 11)
      tree.fit(X,r)

      # 3. h_m
      h_m = tree.predict(X)

      # 4. f_hat에 추가
      f_hat += self.learning_rate * h_m
      self.trees.append(tree)
    
  def predict(self, X):

      f_hat = self.f0

      for m in range(self.n_estimators):
        f_hat += self.learning_rate * self.trees[m].predict(X)
      
      return f_hat

In [ ]:
from sklearn.metrics import mean_squared_error

gbm = GradientBoost(
    n_estimators = 1000,
    learning_rate=0.01,
    max_depth=1
)
gbm.fit(trainX,trainy)
pred = gbm.predict(testX)
gbm_rmse = mean_squared_error(testy, pred, squared = False)
print(f"Custom GBM RMSE:{gbm_rmse:.15f}")

Custom GBM RMSE:56.493210217949098


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from math import sqrt

gbm2 = GradientBoostingRegressor(n_estimators=1000,random_state = 0)
gbm2.fit(trainX,trainy)
pred2 = gbm2.predict(testX)
print(sqrt(mean_squared_error(pred2, testy)))

69.48209229736067


# K-NN


In [ ]:
|from sklearn.datasets import load_iris

idata = load_iris()

print(dir(idata))
print(idata.data.shape)
print(idata.feature_names)
print(idata.target_names)

['DESCR', 'data', 'data_module', 'feature_names', 'filename', 'frame', 'target', 'target_names']
(150, 4)
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']
['setosa' 'versicolor' 'virginica']


In [ ]:
X = idata.data
y = idata.target
trainX, testX, trainy, testy = train_test_split(X, y, test_size=0.2, random_state = 1)
print(trainX.shape)

(120, 4)


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(trainX, trainy)

pred = knn.predict(testX)
print(accuracy_score(testy, pred))

1.0
